In [8]:
import pickle
from knn import ProjectKNN
from xgmodel import ProjectXGBOOST
from mlp import ProjectMLP, MLP

In [4]:
with open('knn.bin', 'rb') as fiile:
    knn = pickle.load(fiile)
knn.test_arx('LocTreino_Equipe_3.csv')

In [6]:
with open('xgboost.bin', 'rb') as fiile:
    xgboost = pickle.load(fiile)
xgboost.test_arx('LocTreino_Equipe_3.csv')

In [9]:
with open('mlp.bin', 'rb') as fiile:
    mlp = pickle.load(fiile)
mlp.test_arx('LocTreino_Equipe_3.csv')

NameError: name 'y_batch' is not defined